In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from google import genai
from google.genai import types
import gradio as gr
load_dotenv()
client = genai.Client()
system_prompt = """
You are a programming help assistant.
Your only task is to answer user queries strictly related to programming, coding, software development, or debugging.
If the user asks anything unrelated to programming, politely decline and remind them that you only answer programming-related questions.

If the user specifies the programming language, provide the solution in that language.
If the user does not specify a programming language, assume the request is for Python and provide a Python example.

Always ensure your answers are:
1. Accurate and concise.
2. Include example code where possible.
3. Include brief explanations to help the user understand.
4. Use proper code formatting.

If a question is unclear, ask for clarification before answering.
"""

def chat_reply(**kwargs):
    user_input = kwargs.get("Code Query", "")

    try:
        response=client.models.generate_content(
             model="gemini-2.5-flash",
             contents = [{ "role": "user", "parts": [{"text": f"{system_prompt}\n\n{user_input}"}] }],
        config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
    )
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        # Handle potential errors during the API call
        print(f"An error occurred: {e}")
        return f"Sorry, I encountered an error trying to get an answer: {e}"
def chat_reply_wrapper(user_input):
    return chat_reply(**{"Code Query": user_input})


with gr.Blocks() as chat:
    input=gr.Textbox(label="Code Query",placeholder="Write your code query (e.g., Python loop example)" )
    output=gr.Textbox(label="AI Explanation",lines=4)
    button=gr.Button("Submit")
    button.click(fn=chat_reply_wrapper,inputs=input,outputs=output ,api_name="Open AI code Explainer")

chat.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
